In [ ]:
import plotly.plotly as py
import cufflinks as cf
import pandas as pd
import geopandas as gp
import numpy as np
import math
import json
import itertools
from pprint import pprint
from datetime import datetime
from dateutil import parser
from IPython.core.interactiveshell import InteractiveShell
import matplotlib
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
#import matplotlib.pyplot as plt
InteractiveShell.ast_node_interactivity = "all"
init_notebook_mode(connected=True)
cf.go_offline()

In [ ]:
londonProfile = pd.read_csv("data/london-borough-profiles.csv")
ambulanceIncidents = pd.read_excel("data/ambulance-borough-monthly.xls", sheet_name='All Ambulance Attended')

In [ ]:
#Data cleaning -- londonProfile

#Keep stats for borough and drop stats for London, England and United Kingdom.
londonProfile = londonProfile.drop([33,34,35,36,37])
#sort by Code
londonProfile = londonProfile.sort_values(by = ['Code'])
#Set code as index, i.e. use E09000001 as index instead of simple number 1,2,3...
londonProfile = londonProfile.set_index(keys='Code')
#show the results -- You can also use print(londonProfile)
londonProfile

#Data cleaning -- ambulanceIncidents
#Drop the column with Total of all Boroughs
ambulanceIncidents = ambulanceIncidents.drop([33])
#sort by Code
ambulanceIncidents = ambulanceIncidents.sort_values(by = ['Code'])
#Set code as index, i.e. use E09000001 as index instead of simple number 1,2,3...
ambulanceIncidents = ambulanceIncidents.set_index(keys='Code')
#show the results
ambulanceIncidents

In [ ]:
ambulanceIncidents['AllIncidents'] = ambulanceIncidents.sum(axis = 1)
ambulanceIncidents

In [ ]:
#Merge our two datasets
lpsmall = londonProfile[['GLA_Population_Estimate_2017', 'GLA_Household_Estimate_2017', 'Inland_Area_(Hectares)', 'Population_density_(per_hectare)_2017', 'Average_Age,_2017', 'Anxiety_score_2011-14_(out_of_10)','Childhood_Obesity_Prevalance_(%)_2015/16']]
mergedDf = pd.merge(ambulanceIncidents[['AllIncidents']], lpsmall, left_index=True, right_index=True)
mergedDf

In [ ]:
mergedDf.corr()

In [ ]:
#Read the shape file
borough = gp.read_file('data/statistical-gis-boundaries-london/MapInfo/London_Borough_Excluding_MHW.tab')
#Change to CRS to UTM
borough = borough.to_crs('+proj=utm +zone=30 +ellps=WGS84 +datum=WGS84 +units=m +no_defs"') # world.to_crs(epsg=3395) would also work
#In this dataset borough code is called GSS_Code so we fix it.
borough = borough.rename(index=str, columns={"GSS_Code": "Code"})
#sort by Code
borough = borough.sort_values(by = ['Code'])
#Set code as index, i.e. use E09000001 as index instead of simple number 1,2,3...
borough = borough.set_index(keys='Code')
#Calculate Centroid of every borough
borough['centroid'] = borough['geometry'].centroid
print(borough)
borough.total_bounds

In [ ]:
ax = borough.plot()

In [ ]:
londonMinX = borough.total_bounds[0]
londonMinY = borough.total_bounds[1]
londonMaxX = borough.total_bounds[2]
londonMaxY = borough.total_bounds[3]

print('London Length (in km):', (londonMaxX - londonMinX)/1000)
print('London Breadth (in km):', (londonMaxY - londonMinY)/1000)
#Divide london in cells of 500m * 500m
# cellCentres = np.zeros([58*2+1, 45*2 + 1])
# cellCentres.shape
#Total possible location = Breadth * length = 91 * 117 = 10647
# Row corresponds to breadth and length to coloums. 
grid_coloums = 58*2+1
grid_rows = 45*2 + 1
M = grid_rows * grid_coloums

# Total number of possible cells 
M

In [ ]:
finalDataFrame = pd.merge(borough[['centroid']], mergedDf, left_index=True, right_index=True)
finalDataFrame


In [ ]:
#Function related to the distance between points, cells etc
import math
# A function that returns the index of the cell where a point will lie
def pointToCellIdConversion(x,y):
    coloumn = int((x - londonMinX) / 500)
    row = int((y - londonMinY) / 500)
    cid = row * grid_coloums + coloumn
    return (cid)

# A function that returns the centre of the cell with a given index
def cellIdtoPointConversion(cid):
    row = int(cid / grid_coloums)
    coloumn = int(cid % grid_coloums)
    return indexToPointConversion(row,coloumn)

# A function that returns the centre of the cell with a given index
def indexToPointConversion(row,coloumn):
    y = londonMinY + 250 + row * 500
    x = londonMinX + 250 + coloumn * 500
    return(x,y)

def l1DistanceBetweenCellIdandPoint(cid, x1, y1):
    x2,y2 = cellIdtoPointConversion(cid)
    return( math.fabs(x2-x1) +  math.fabs(y2-y1))

def l2DistanceBetweenCellIdandPoint(cid, x1, y1):
    x2,y2 = cellIdtoPointConversion(cid)
    return math.sqrt((x2-x1)*(x2-x1) +  (y2-y1)*(y2-y1))
# Test the above functions
borough.total_bounds
cellIdtoPointConversion(0)
cellIdtoPointConversion(117)
cellIdtoPointConversion(5670)
pointToCellIdConversion(673171.2412707923, 5685885.993756533) # First row first coloumn
pointToCellIdConversion(673171.2412707923, 5686385.993756533) # Next row first coloumn
l1DistanceBetweenCellIdandPoint(117, 673171.2412707923, 5685885.993756533) # They are 500 m appart
l2DistanceBetweenCellIdandPoint(117, 673171.2412707923, 5685885.993756533) # They are 500 m appart

In [ ]:
# Define parameters for genetic algorithm

#Number of ambulance stations or the genes
ambulance_stations = 20

#chromosome_count in the population (we take fixed size population)
chromosome_count = 100 

#Probability that any "one" gene will mutate after the crossover
mutation_probability = 0.50 

# Total number of possible ambulance sites (already calculated)
total_possible_ambulance_sites = M

In [ ]:
#Genetic algorithm!!

# Fitness function for a cell/gene
def fitnessFunction(cid):
    num,deno = 0,0
    for index, row in finalDataFrame.iterrows():
        distance = l1DistanceBetweenCellIdandPoint(cid, row['centroid'].x, row['centroid'].y )
        num += row['AllIncidents'] * distance
        deno += row['AllIncidents']
    return (num/deno)

# Fitness function for a whole Chromosome
def fitnessFunctionChromosome(chromosome):
    if(chromosome.ndim > 1):
        print("Chromosome should be 1D!!! \n Something is wrong")
        return(sys.maxsize)
    else:
        score = 0
        for g in chromosome:
            score += fitnessFunction(g)
    return(score)

# Crossover 
def chromosomeCrossover(c1,c2):
    c = np.unique(np.concatenate((c1,c2)))
    return np.random.choice(c, ambulance_stations, replace=False)
    
# Mutation 
def mutation(c):
    r = np.random.rand()
    #Mutation time!!
    if (r < mutation_probability):
        s = np.random.randint(0,ambulance_stations,1)[0]
        newLocation = np.random.randint(0,total_possible_ambulance_sites,1)[0]
        #Make sure we are not selecting one of the already selected sites
        while(np.isin([newLocation],c)[0]):
              newLocation = np.random.randint(0,total_possible_ambulance_sites,1)[0]
        c[s] = newLocation
    return c

def populationInitialization(c,g):
    population = np.random.randint(0,total_possible_ambulance_sites,(c,g),dtype='int')        
    return(population)
   
def populationScore(pop):
    nrows =  pop.shape[0]
    score = np.zeros((nrows),dtype='int')
    for i in range(0,nrows):
        score[i] = fitnessFunctionChromosome(population[i])
    return (score)

# In every gerenation top "perc" percetage of the population is retained and the rest is generated from the top perc using crossovers
def newPopulation(pop, perc = 0.10):
    sc = populationScore(population)
    top_count = int(chromosome_count * perc)
    top_index = np.argpartition(sc, top_count)[0:top_count]
    newPopulation = np.zeros((chromosome_count,ambulance_stations),dtype='int')
    # New population is top perc of old population and crossovers
    for i in range(top_count):
        newPopulation[i] = population[top_index[i]]
    pitr = range(top_count)
    for i in range(top_count,chromosome_count):
        parents = np.random.choice(pitr, 2, replace=False)
        child = chromosomeCrossover(newPopulation[parents[0]],newPopulation[parents[1]])
        newPopulation[i] = mutation(child)
    return(newPopulation)

In [ ]:
# We define a fixed population of 100 individuals/chromosomes with k genes each
# in every iteration the 90% of the population is replaced with the offsprings

min_scores =[]
num_iter = 20
population = populationInitialization(chromosome_count,ambulance_stations)
best_solution = []

for i in range(20):
    sc = populationScore(population)
    print("Itr:", i)
    print("Min Score",min(sc))
    min_scores.append(min(sc))
    # Best solution in this generation
    ind = np.unravel_index(np.argmin(sc, axis=None), sc.shape)
    best_solution = population[ind]
    population = newPopulation(population, 0.10)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(min_scores)
# plt.show()